In [ ]:
# default_exp dbt_cellmagic

# DBT Cell Magic 

> provides the `%%dbt` cell magic for creating models and analyses 

In [ ]:
# hide
# notest
%reload_ext autoreload
%reload_ext lab_black
%autoreload 2

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# exporti

import IPython
from IPython.core import magic_arguments
from IPython.core.magic import register_cell_magic, register_line_magic

In [ ]:
# exporti
from pathlib import Path
from typing import Union, List, Dict, Optional

In [ ]:
# exporti
from fastcore.all import patch

In [ ]:
# exporti
from dbt.main import parse_args, adapter_management
from dbt.task.compile import CompileTask
from dbt.contracts.results import RunExecutionResult
import dbt.flags
import dbt.tracking

In [ ]:
# exporti
dbt.tracking.active_user = dbt.tracking.User(None)
dbt.flags.INDIRECT_SELECTION = "eager"
dbt.flags.WRITE_JSON = True
dbt.flags.QUIET = True  # silent

In [ ]:
# exporti

from fal import FalDbt
import faldbt.lib as fallib

In [ ]:
# exporti

import pandas as pd

In [ ]:
# export


class DbtMagicObject:
    def __init__(
        self,
        raw_sql: str,  # sql string
        file: str,  # path to sql file (relative to dbt project dir)
        project_dir: Union[str, Path],  # dbt project dir
        notebook_name: Optional[str] = None,  # name of notebook
        profile_dir: Union[str, Path] = "~/.dbt",  # dbt profiles dir
    ):
        """Create a holder of dbt cell magic parameters"""
        self.raw_sql = raw_sql
        self.file = file
        self.project_dir = (
            Path(project_dir) if type(project_dir) == str else project_dir
        )
        self.notebook_name = notebook_name
        self.profile_dir = (
            Path(profile_dir) if type(profile_dir) == str else profile_dir
        )

        # dynamic runtime attributes
        self._formatted_sql: Optional[str] = None
        self._compile_result: Optional[RunExecutionResult] = None
        self._compiled_sql: Optional[str] = None
        self._df_result: Optional[pd.DataFrame] = None

In [ ]:
# export
@patch
def _reformat_sql(self: DbtMagicObject) -> None:
    """Add auto generated notice for nbdbt generated sql"""
    if not self.notebook_name:
        contents = "-- AUTOGENERATED! DO NOT EDIT!\n" + self.raw_sql
    else:
        contents = (
            f"-- AUTOGENERATED! DO NOT EDIT! File to edit: {self.notebook_name} (unless otherwise specified).\n"
            + self.raw_sql
        )
    self._formatted_sql = contents

In [ ]:
# export
@patch
def _write_sql(self: DbtMagicObject) -> None:
    """Write sql to file"""
    path = self.project_dir / self.file
    self._reformat_sql()
    with open(path, "w") as f:
        f.write(self._formatted_sql)

In [ ]:
# export
@patch
def _compile_model(self: DbtMagicObject) -> None:
    """Compile model and store compile result"""
    parsed = parse_args(
        ["compile", "--select", self.file, "--project-dir", str(self.project_dir)]
    )
    with adapter_management():
        task = CompileTask.from_args(args=parsed)
        self._compile_result = task.run()
        self._compiled_sql = self._compile_result.results[0].node.compiled_sql

In [ ]:
# export
@patch
def _exec_faldbt_ref(self: DbtMagicObject) -> None:
    """Execute sql and return df"""
    if self._compiled_sql is None:
        raise ValueError("Model compilation step has not been executed")

    faldbt = FalDbt(
        profiles_dir=str(self.profile_dir), project_dir=str(self.project_dir)
    )
    profile_target = faldbt._profile_target
    # adapter_response, result
    _, result = fallib._execute_sql(
        str(self.project_dir), str(self.profile_dir), self._compiled_sql, profile_target
    )
    self._exec_sql_result = result
    df_result = pd.DataFrame.from_records(
        result.table.rows, columns=result.table.column_names, coerce_float=True
    )
    self._df_result = df_result

In [ ]:
# export
@patch
def ref(self: DbtMagicObject) -> pd.DataFrame:
    self._exec_faldbt_ref()
    return self._df_result

In [ ]:
# export
@magic_arguments.magic_arguments()
@magic_arguments.argument(
    "-a",
    "--assign",
    type=str,
    default=None,
    help=("If provided, save the output to this variable instead of displaying it."),
)
@magic_arguments.argument(
    "-p",
    "--project",
    type=str,
    help=("dbt project directory"),
)
@magic_arguments.argument(
    "-n",
    "--notebook",
    type=str,
    default=None,
    help=("notebook source file"),
)
@magic_arguments.argument(
    "file",
    type=str,
    help=("file path to write to"),
)
@register_cell_magic("dbt")
def write_dbt(line, cell):
    try:
        from dbt.main import parse_args
    except ImportError:
        return "'dbt-core' not installed. Did you run 'pip install dbt-core'?"
    line_args = magic_arguments.parse_argstring(write_dbt, line)
    dmo = DbtMagicObject(cell, line_args.file, line_args.project, line_args.notebook)
    dmo._write_sql()
    dmo._compile_model()
    if line_args.assign:
        IPython.get_ipython().push({line_args.assign: dmo})
        return None
    results = dmo.ref()
    return results

In [ ]:
# hide
test_project = "my_dbt_project"
test_project_dir = f"../{test_project}"
test_model1 = "analyses/sample1.sql"
test_model2 = "analyses/sample2.sql"

In [ ]:
# hide
!rm -f {test_project_dir}/{test_model1}
!rm -f {test_project_dir}/target/compiled/{test_project}/{test_model1}

In [ ]:
%%dbt -p ../my_dbt_project analyses/sample1.sql

select *
from {{ ref('my_first_dbt_model') }}
where id is not null

,id
0,1.0


In [ ]:
# hide
assert _.equals(pd.DataFrame(data={"id": [1.0]}))

In [ ]:
# hide
# check that analyses file was created
assert (Path(test_project_dir) / test_model1).exists()

In [ ]:
# hide
# check that compiled file was created
assert (
    Path(test_project_dir) / "target" / "compiled" / test_project / test_model1
).exists()

In [ ]:
# hide
!rm -f {test_project_dir}/{test_model2}
!rm -f {test_project_dir}/target/compiled/{test_project}/{test_model2}

In [ ]:
if "sample2" in globals():
    del sample2

In [ ]:
%%dbt -a sample2 -p ../my_dbt_project -n notebooks/00_dbt_cellmagic.ipynb analyses/sample2.sql 

select *
from {{ ref('my_second_dbt_model') }}
where id is not null

In [ ]:
# hide
# check that analyses file was created
assert (Path(test_project_dir) / test_model2).exists()

In [ ]:
# hide
# check that compiled file was created
(Path(test_project_dir) / "target" / "compiled" / test_project / test_model2).exists()

True

In [ ]:
# hide
assert "sample2" in globals()

In [ ]:
results = sample2.ref()

In [ ]:
results

,id
0,1.0


In [ ]:
# hide
assert results.equals(pd.DataFrame(data={"id": [1.0]}))